# 1. Importing Libraries

In [4]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import StratifiedKFold

import os

### Testing Tensorflow GPU

In [5]:
tf.test.is_built_with_cuda()

True

# 2. Project Variables

In [6]:
DATA_DIR = '../train-test-data'
NUM_FOLDS = 10
TASKS_TO_RUN = ['2aii', '2aiii']

# 3. Model Training

In [7]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, label, shuffle=True, batch_size=8):
    dataframe = dataframe.copy()
    dataframe['target'] = np.where(dataframe[label]=='INCREASED RISK', 1, 0)
    dataframe = dataframe.drop(columns=label)
    
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [25]:
def train_kfold(task, included_cols):
    # Read train csv
    train_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_train.csv'), index_col=0)

    # Metric arrays
    acc_per_fold = []
    loss_per_fold = []
    
    kfold = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
    fold_no = 1
    for train_idx, val_idx in kfold.split(train_df.drop(task, axis=1), train_df[[task]]):
        train = train_df.iloc[train_idx] 
        test = train_df.iloc[val_idx]

        # Generate feauture columns
        feature_columns = []
        for col in included_cols:
            feature_columns.append(tf.feature_column.numeric_column(col))

        # Generating a tensorflow dataset
        train_ds = df_to_dataset(train, task)
        test_ds = df_to_dataset(test, task)

        # Building model
        model = tf.keras.Sequential([
            tf.keras.layers.DenseFeatures(feature_columns),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1)
        ])

        model.compile(optimizer='adam',
                             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                             metrics=['accuracy'])
        
        # Fitting Model
        history = model.fit(train_ds, 
                            epochs=10, 
                            verbose=1)

        # Evaluate Model
        scores = model.evaluate(test_ds, verbose=0)
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
        
        fold_no += 1
    
    metrics = {
        'ACCURACY': {
            'ALL': acc_per_fold,
            'MEAN': np.mean(acc_per_fold),
            'STDEV': np.std(acc_per_fold)
        }
    }
    return metrics

In [26]:
metrics = {}

included_cols = ['CHILD_SEX','IDD_SCORE','AGE','HHID_count','HH_AGE','FOOD_EXPENSE_WEEKLY',
                 'NON-FOOD_EXPENSE_WEEKLY','HDD_SCORE','FOOD_INSECURITY','YoungBoys','YoungGirls',
                 'AverageMonthlyIncome','BEN_4PS','AREA_TYPE','FOOD_EXPENSE_WEEKLY_pc',
                 'NON-FOOD_EXPENSE_WEEKLY_pc','AverageMonthlyIncome_pc']

for task in TASKS_TO_RUN:
    metric = train_kfold(task, included_cols)
    metrics[task] = metric

Epoch 1/10
34/34 [==============================] - 1s 5ms/step - loss: 32.8526 - accuracy: 0.6949
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 17.9291 - accuracy: 0.7169
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: 9.1627 - accuracy: 0.6912
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 9.2728 - accuracy: 0.7316
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 8.9484 - accuracy: 0.6912
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 10.8257 - accuracy: 0.7978
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 9.3887 - accuracy: 0.6949
Epoch 8/10
34/34 [==============================] - 0s 5ms/step - loss: 6.9591 - accuracy: 0.6838
Epoch 9/10
34/34 [==============================] - 0s 4ms/step - loss: 10.8298 - accuracy: 0.7463
Epoch 10/10
34/34 [==============================] - 0s 4ms/step - loss: 7.6395 - accuracy: 0.7390
Epoch 1/10


34/34 [==============================] - 1s 5ms/step - loss: 31.4411 - accuracy: 0.7463
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 21.6666 - accuracy: 0.7500
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: 20.1754 - accuracy: 0.7022
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 19.6315 - accuracy: 0.7096
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 10.4085 - accuracy: 0.7169
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 5.0172 - accuracy: 0.7426
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 11.9420 - accuracy: 0.7353
Epoch 8/10
34/34 [==============================] - 0s 5ms/step - loss: 9.9070 - accuracy: 0.7316
Epoch 9/10
34/34 [==============================] - 0s 5ms/step - loss: 8.1626 - accuracy: 0.7463
Epoch 10/10
34/34 [==============================] - 0s 4ms/step - loss: 6.5428 - accuracy: 0.7206
Epoch 1/10
34/34 [======

35/35 [==============================] - 1s 4ms/step - loss: 113.5440 - accuracy: 0.6593
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 19.5363 - accuracy: 0.7070
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 5.7849 - accuracy: 0.7363
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 6.2204 - accuracy: 0.7216
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 6.4193 - accuracy: 0.7546
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 6.3736 - accuracy: 0.7399
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 10.9562 - accuracy: 0.6886
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 9.9565 - accuracy: 0.7692
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 5.0331 - accuracy: 0.7179
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 9.0160 - accuracy: 0.7582
Epoch 1/10
35/35 [========

Epoch 1/10
35/35 [==============================] - 1s 4ms/step - loss: 26.7391 - accuracy: 0.7289
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 10.4269 - accuracy: 0.6850
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 9.1031 - accuracy: 0.7619
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 17.5840 - accuracy: 0.7582
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 21.5790 - accuracy: 0.7363
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 24.5107 - accuracy: 0.7326
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 12.7845 - accuracy: 0.7326
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 8.0243 - accuracy: 0.6923
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 6.9524 - accuracy: 0.7509
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 4.5496 - accuracy: 0.7473
Epoch 1/10


35/35 [==============================] - 1s 5ms/step - loss: 11.3239 - accuracy: 0.6850
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 7.5537 - accuracy: 0.7436
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 14.6465 - accuracy: 0.7363
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 6.9000 - accuracy: 0.7106
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 4.6337 - accuracy: 0.7546
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 9.9496 - accuracy: 0.7399
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 8.7302 - accuracy: 0.7436
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 6.6491 - accuracy: 0.7619
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 10.0530 - accuracy: 0.7692
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 6.6934 - accuracy: 0.7363
Epoch 1/10
35/35 [=========

...

34/34 [==============================] - 1s 4ms/step - loss: 63.8131 - accuracy: 0.6397
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 14.0692 - accuracy: 0.7206
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: 17.0636 - accuracy: 0.5846
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 34.1778 - accuracy: 0.6949
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 16.3815 - accuracy: 0.7316
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 18.6013 - accuracy: 0.6103
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 9.9857 - accuracy: 0.6728
Epoch 8/10
34/34 [==============================] - 0s 4ms/step - loss: 16.3248 - accuracy: 0.6949
Epoch 9/10
34/34 [==============================] - 0s 4ms/step - loss: 18.9639 - accuracy: 0.7132
Epoch 10/10
34/34 [==============================] - 0s 4ms/step - loss: 21.2669 - accuracy: 0.6618
Epoch 1/10
34/34 [===

35/35 [==============================] - 1s 4ms/step - loss: 60.2176 - accuracy: 0.6777
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 36.0026 - accuracy: 0.6667
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 21.0635 - accuracy: 0.6740
Epoch 4/10
35/35 [==============================] - 0s 5ms/step - loss: 11.4371 - accuracy: 0.6813
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 24.2689 - accuracy: 0.7143
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 19.1926 - accuracy: 0.6520
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 10.9155 - accuracy: 0.6923
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 6.0586 - accuracy: 0.6923
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 19.8954 - accuracy: 0.6447
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 12.9648 - accuracy: 0.6850
Epoch 1/10
35/35 [===

Epoch 1/10
35/35 [==============================] - 1s 4ms/step - loss: 23.0570 - accuracy: 0.6593
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 35.6565 - accuracy: 0.6081
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 19.7511 - accuracy: 0.7070
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 9.3718 - accuracy: 0.6227
Epoch 5/10
35/35 [==============================] - 0s 5ms/step - loss: 14.5535 - accuracy: 0.6447
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 10.8124 - accuracy: 0.6667
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 19.5040 - accuracy: 0.6410
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 16.3816 - accuracy: 0.6813
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 8.6278 - accuracy: 0.7143
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 12.3989 - accuracy: 0.6227
Epoch 1/10


35/35 [==============================] - 1s 4ms/step - loss: 32.4757 - accuracy: 0.6447
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 27.2790 - accuracy: 0.6923
Epoch 3/10
35/35 [==============================] - 0s 5ms/step - loss: 27.7382 - accuracy: 0.6777
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 25.8257 - accuracy: 0.6703
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 16.2170 - accuracy: 0.6740
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 23.1343 - accuracy: 0.6703
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 22.8825 - accuracy: 0.6300
Epoch 8/10
35/35 [==============================] - 0s 5ms/step - loss: 23.0926 - accuracy: 0.6740
Epoch 9/10
35/35 [==============================] - 0s 5ms/step - loss: 17.8513 - accuracy: 0.6703
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 23.1739 - accuracy: 0.6520
Epoch 1/10
35/35 [==

35/35 [==============================] - 1s 4ms/step - loss: 54.3351 - accuracy: 0.6337
Epoch 2/10
35/35 [==============================] - 0s 4ms/step - loss: 28.0038 - accuracy: 0.6886
Epoch 3/10
35/35 [==============================] - 0s 4ms/step - loss: 18.6567 - accuracy: 0.6520
Epoch 4/10
35/35 [==============================] - 0s 4ms/step - loss: 13.8754 - accuracy: 0.6264
Epoch 5/10
35/35 [==============================] - 0s 4ms/step - loss: 14.3093 - accuracy: 0.6813
Epoch 6/10
35/35 [==============================] - 0s 4ms/step - loss: 11.0990 - accuracy: 0.6593
Epoch 7/10
35/35 [==============================] - 0s 4ms/step - loss: 12.0859 - accuracy: 0.6813
Epoch 8/10
35/35 [==============================] - 0s 4ms/step - loss: 14.5831 - accuracy: 0.6996
Epoch 9/10
35/35 [==============================] - 0s 4ms/step - loss: 20.2568 - accuracy: 0.7106
Epoch 10/10
35/35 [==============================] - 0s 4ms/step - loss: 9.2544 - accuracy: 0.5971
Epoch 1/10
35/35 [===

In [27]:
for task in TASKS_TO_RUN:
    print(f'{task}: {metrics[task]["ACCURACY"]["MEAN"]}')

2aii: 71.97849422693253
2aiii: 74.55913960933685


# Model Evaluation

### Note: This runs evaluates the models with the testing set. Run only at the end.

In [28]:
def train_and_test(task, included_cols, models):
    train_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_train.csv'), index_col=0)
    test_df = pd.read_csv(os.path.join(DATA_DIR, f'{task}_test.csv'), index_col=0)
    
    # Generate feauture columns
    feature_columns = []
    for col in included_cols:
        feature_columns.append(tf.feature_column.numeric_column(col))

    # Generating a tensorflow dataset
    train_ds = df_to_dataset(train_df, task)
    test_ds = df_to_dataset(test_df, task)

    # Building model
    models[task] = tf.keras.Sequential([
        tf.keras.layers.DenseFeatures(feature_columns),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    models[task].compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Fitting Model
    history = models[task].fit(train_ds, 
                        epochs=10, 
                        verbose=1)

    # Evaluate Model
    scores = models[task].evaluate(test_ds, verbose=0)
    
    metrics = {
        'ACCURACY': scores[1]*100
    }
    return metrics

In [29]:
models = {}
metrics = {}
included_cols = ['CHILD_SEX','IDD_SCORE','AGE','HHID_count','HH_AGE','FOOD_EXPENSE_WEEKLY',
                 'NON-FOOD_EXPENSE_WEEKLY','HDD_SCORE','FOOD_INSECURITY','YoungBoys','YoungGirls',
                 'AverageMonthlyIncome','BEN_4PS','AREA_TYPE','FOOD_EXPENSE_WEEKLY_pc',
                 'NON-FOOD_EXPENSE_WEEKLY_pc','AverageMonthlyIncome_pc']

for task in TASKS_TO_RUN:
    metric = train_and_test(task, included_cols, models)
    metrics[task] = metric

Epoch 1/10
38/38 [==============================] - 1s 4ms/step - loss: 32.4853 - accuracy: 0.7360
Epoch 2/10
38/38 [==============================] - 0s 4ms/step - loss: 13.2367 - accuracy: 0.7228
Epoch 3/10
38/38 [==============================] - 0s 4ms/step - loss: 11.1162 - accuracy: 0.7525
Epoch 4/10
38/38 [==============================] - 0s 4ms/step - loss: 9.0873 - accuracy: 0.7129
Epoch 5/10
38/38 [==============================] - 0s 4ms/step - loss: 10.5430 - accuracy: 0.7228
Epoch 6/10
38/38 [==============================] - 0s 4ms/step - loss: 9.4680 - accuracy: 0.7294
Epoch 7/10
38/38 [==============================] - 0s 4ms/step - loss: 11.6647 - accuracy: 0.7195
Epoch 8/10
38/38 [==============================] - 0s 4ms/step - loss: 6.0390 - accuracy: 0.7426
Epoch 9/10
38/38 [==============================] - 0s 4ms/step - loss: 10.0519 - accuracy: 0.7030
Epoch 10/10
38/38 [==============================] - 0s 4ms/step - loss: 7.6912 - accuracy: 0.7591
Epoch 1/10


38/38 [==============================] - 1s 4ms/step - loss: 65.8600 - accuracy: 0.6535
Epoch 2/10
38/38 [==============================] - 0s 4ms/step - loss: 16.5101 - accuracy: 0.6601
Epoch 3/10
38/38 [==============================] - 0s 4ms/step - loss: 15.4095 - accuracy: 0.6964
Epoch 4/10
38/38 [==============================] - 0s 4ms/step - loss: 7.6158 - accuracy: 0.6766
Epoch 5/10
38/38 [==============================] - 0s 4ms/step - loss: 24.0474 - accuracy: 0.6370
Epoch 6/10
38/38 [==============================] - 0s 4ms/step - loss: 9.5568 - accuracy: 0.6733
Epoch 7/10
38/38 [==============================] - 0s 4ms/step - loss: 6.7820 - accuracy: 0.6568
Epoch 8/10
38/38 [==============================] - 0s 4ms/step - loss: 8.2400 - accuracy: 0.6502
Epoch 9/10
38/38 [==============================] - 0s 4ms/step - loss: 14.8450 - accuracy: 0.6436
Epoch 10/10
38/38 [==============================] - 0s 4ms/step - loss: 15.2021 - accuracy: 0.6964


In [30]:
for task in TASKS_TO_RUN:
    print(f'{task}: {metrics[task]["ACCURACY"]}')

2aii: 69.63696479797363
2aiii: 78.54785323143005
